In [1]:
outdir <- "/data2/mito_lineage/Analysis/annotation/output/data"
prefix <- "CHIP_april08_2021_Croker/MTblacklist/"
singlecell_sumstats_dir <- "/data2/isshamie/mito_lineage/data/processed/mtscATAC/CHIP_april08_2021_Croker/MTblacklist/"
cellr_indir <- "/data2/isshamie/mito_lineage/data/processed/mtscATAC/CHIP_april08_2021_Croker/MTblacklist"
external_dat_dir <- "/data2/mito_lineage/Analysis/annotation/output/data/granja_cd34"
external_frag_file <- "granja_cd34.fragments.tsv"
external_prefix <- "GSE129785_scATAC-Hematopoiesis-CD34"

In [2]:
fragment_file <- file.path(external_dat_dir, external_frag_file)

In [3]:
genome_dir <- outdir
outdir <- file.path(outdir, prefix)
#dir.create(outdir, recursive=TRUE, showWarnings = TRUE)

In [4]:
if (FALSE){
    BiocManager::install("EnsDb.Hsapiens.v75")
    }

In [5]:
library(EnsDb.Hsapiens.v75)

library(Signac)
library(Seurat)
library(data.table)
library(dplyr)

library(umap)

Loading required package: ensembldb

Loading required package: BiocGenerics

Loading required package: parallel


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:parallel’:

    clusterApply, clusterApplyLB, clusterCall, clusterEvalQ,
    clusterExport, clusterMap, parApply, parCapply, parLapply,
    parLapplyLB, parRapply, parSapply, parSapplyLB


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, table, tapply,
    union, unique, unsplit, which.max, which.min


Loading required package: GenomicRanges

Loading required package: stats4

Loa

In [6]:
if(FALSE){
  library(GenomeInfoDb)
  library(EnsDb.Hsapiens.v75)
  
  # extract gene coordinates from Ensembl, and ensure name formatting is consistent with Seurat object 
  gene.coords <- genes(EnsDb.Hsapiens.v75, filter = ~ gene_biotype == "protein_coding")
  seqlevelsStyle(gene.coords) <- 'UCSC'
  genebody.coords <- keepStandardChromosomes(gene.coords, pruning.mode = 'coarse')
  genebodyandpromoter.coords <- Extend(x = gene.coords, upstream = 2000, downstream = 0)
  saveRDS(genebodyandpromoter.coords, file.path(genome_dir, "genebodyandpromoter.coords_EnsDb.Hsapiens.v75.rds"))
}else{
genebodyandpromoter.coords <- readRDS(file.path(genome_dir, "genebodyandpromoter.coords_EnsDb.Hsapiens.v75.rds"))
}

# Compute for external data

In [ ]:
# peaks_cd34 <-diffloop::bedToGRanges(peaks_cd34_f)

# countdf <- data.frame(peaks = S4Vectors::queryHits(ovPEAK),
#                     sample = as.numeric(id)[S4Vectors::subjectHits(ovPEAK)]) %>% 
#             dplyr::group_by(peaks,sample) %>% dplyr::summarise(count = n()) %>% data.matrix()

# m <- Matrix::sparseMatrix(i = c(countdf[,1], length(peaks_external)),
#                         j = c(countdf[,2], length(barcodes_found)),
#                         x = c(countdf[,3],0))



# readMM <- Matrix::readMM(file.path(external_dat_dir, external_frag_file))
# readMM

In [ ]:
# peaks <- fread(file.path(external_dat_dir, "GSE129785_scATAC-Hematopoiesis-CD34.peaks.bed")) %>%
#     data.frame() %>% setnames(c('chr', 'start', 'end')) 
# peaks

# peaks_names <- apply(peaks, 1, FUN=function(x) paste(x['chr'], ':', x['start'], "-", x['end'], collapse=""))
# peaks_names

In [7]:
metadata <- read.csv(
  file = file.path(external_dat_dir, paste0(external_prefix, ".cell_barcodes.txt")),
  header = TRUE,
  sep='\t',
)
rownames(metadata) <- metadata$Group_Barcode
cell_names <- rownames(metadata)
metadata

,UMAP1,UMAP2,Clusters,Group,depth,FRIP,Barcodes,Internal_Name,Group_Barcode
,<dbl>,<dbl>,<chr>,<chr>,<int>,<dbl>,<chr>,<chr>,<chr>
Bone_Marrow_Rep1#CAAGAAAGTCAAGACG-1,-2.419656,7.603472,Cluster6,Bone_Marrow_Rep1,29301,0.3471895,CAAGAAAGTCAAGACG-1,SUHealthy_BM_B1_50,Bone_Marrow_Rep1#CAAGAAAGTCAAGACG-1
Bone_Marrow_Rep1#AAAGGGCAGTACCCAT-1,-7.153399,-5.512960,Cluster11,Bone_Marrow_Rep1,29608,0.4491860,AAAGGGCAGTACCCAT-1,SUHealthy_BM_B1_52,Bone_Marrow_Rep1#AAAGGGCAGTACCCAT-1
Bone_Marrow_Rep1#CACAACATCGTGGGTC-1,-8.475636,-5.377475,Cluster9,Bone_Marrow_Rep1,16810,0.4986318,CACAACATCGTGGGTC-1,SUHealthy_BM_B1_53,Bone_Marrow_Rep1#CACAACATCGTGGGTC-1
Bone_Marrow_Rep1#TTACCCGTCTGATCCC-1,-6.706686,12.041433,Cluster2,Bone_Marrow_Rep1,47369,0.3847031,TTACCCGTCTGATCCC-1,SUHealthy_BM_B1_55,Bone_Marrow_Rep1#TTACCCGTCTGATCCC-1
Bone_Marrow_Rep1#CTTTGCGGTACGGTTT-1,-7.591989,-3.001234,Cluster9,Bone_Marrow_Rep1,23342,0.4957159,CTTTGCGGTACGGTTT-1,SUHealthy_BM_B1_56,Bone_Marrow_Rep1#CTTTGCGGTACGGTTT-1
Bone_Marrow_Rep1#GGTTGCGCAGGGAGTT-1,-6.898403,11.516414,Cluster2,Bone_Marrow_Rep1,78378,0.4175853,GGTTGCGCAGGGAGTT-1,SUHealthy_BM_B1_57,Bone_Marrow_Rep1#GGTTGCGCAGGGAGTT-1
Bone_Marrow_Rep1#GCTCAGGTCAGAATGA-1,-9.729723,-4.425398,Cluster9,Bone_Marrow_Rep1,27976,0.4422005,GCTCAGGTCAGAATGA-1,SUHealthy_BM_B1_61,Bone_Marrow_Rep1#GCTCAGGTCAGAATGA-1
Bone_Marrow_Rep1#TTGCGAACAGGGTACA-1,-6.445809,12.357817,Cluster2,Bone_Marrow_Rep1,12076,0.3103263,TTGCGAACAGGGTACA-1,SUHealthy_BM_B1_73,Bone_Marrow_Rep1#TTGCGAACAGGGTACA-1
Bone_Marrow_Rep1#CGGACCACAAGCAACG-1,-7.710368,-4.121327,Cluster9,Bone_Marrow_Rep1,24873,0.4631930,CGGACCACAAGCAACG-1,SUHealthy_BM_B1_75,Bone_Marrow_Rep1#CGGACCACAAGCAACG-1


In [ ]:
# #fragments <- read.table(fragment_file)

# fragments <- read.csv(
#       file = fragment_file,
#       header = FALSE,
#       sep='\t')

# fragments

In [8]:
library('Matrix')


Attaching package: 'Matrix'


The following object is masked from 'package:S4Vectors':

    expand




In [9]:
library(magrittr)
library(readr)
library(Matrix)
library(tidyr)
library(dplyr)



Attaching package: 'magrittr'


The following object is masked from 'package:AnnotationFilter':

    not



Attaching package: 'tidyr'


The following object is masked from 'package:magrittr':

    extract


The following objects are masked from 'package:Matrix':

    expand, pack, unpack


The following object is masked from 'package:S4Vectors':

    expand




In [ ]:

# peak-bc matrix
mex_dir_path <- "/opt/sample345/outs/filtered_peak_bc_matrix"
mtx_path <- file.path(external_dat_dir, paste0(external_prefix, ".mtx"))
feature_path <- file.path(external_dat_dir, paste0(external_prefix, ".peaks.bed"))
barcode_path <- file.path(external_dat_dir, paste0(external_prefix, ".cell_barcodes.txt"))
 
features <- readr::read_tsv(feature_path, col_names = F) %>% tidyr::unite(feature)
barcodes <- readr::read_tsv(barcode_path, col_names = F) %>% tidyr::unite(barcode)
 



In [ ]:
mtx <- Matrix::readMM(mtx_path) %>%
  magrittr::set_rownames(features$feature) %>%
  magrittr::set_colnames(metadata$Group_Barcode)


In [ ]:
features <- readr::read_tsv(feature_path, col_names = F) %>% tidyr::unite(feature)


In [ ]:
raw_features <- readr::read_tsv(feature_path, col_names = F)
raw_features

In [ ]:
features <- paste(raw_features[,c('X1')], ":", 
                  raw_features[,c('X2')], '-',  
                  raw_features[,c('X3')])
features

In [ ]:
tidyr::unite(raw_features[,c("X1","X2")],sep=":")
#raw_features

In [ ]:
features

In [ ]:
mtx <- Matrix::readMM(mtx_path) %>%
  magrittr::set_rownames(features$feature) %>%
  magrittr::set_colnames(metadata$Group_Barcode)



In [ ]:
mtx

In [ ]:
# #counts <- Matrix::readMM(file.path(external_dat_dir, paste0(external_frag_file, ".counts.mtx")))
# counts <- fread(
#           file = file.path(external_dat_dir, paste0(external_frag_file, ".counts.mtx")),
#           header = FALSE,
#           sep='\t')
# counts

# pos <- unique(counts$V1)
# cell <- unique(counts$V2)

# cell_dt = data.table(ID=cell, cell=cell, cell_ind=1:length(cell))
# pos_dt = data.table(ID=pos, pos=pos, pos_ind=1:length(pos))
# pos_dt

# counts = merge(counts, pos_dt, by.x="V1", by.y="pos")


# counts = merge(counts, cell_dt, by.x="V2", by.y="cell")


# counts

In [ ]:
library(mltools)

In [ ]:
# counts_dg <- sparseMatrix(i=counts$pos_ind, j=counts$cell_ind, x=counts$V3,
#                          dims=c(dim(pos_dt)[1], dim(cell_dt)[1]),
#                          dimnames=c(pos_dt$pos, cell_dt$cell)) #as(counts, 'dgCMatrix')
# counts_dg

In [ ]:
mtx

In [ ]:
isna_inds <- which(is.null(rownames(mtx)))
isna_inds

In [ ]:
chrom_assay <- CreateChromatinAssay(
  counts = mtx,
  sep = c(":", "-"),
  genome = 'hg38',
  fragments = fragment_file,
  min.cells = 10,
  min.features = 200
)


In [ ]:

cd34 <- CreateSeuratObject(
  counts = chrom_assay,
  assay = "peaks",
  meta.data = metadata
)

In [ ]:
chrom_assay

In [ ]:
sessionInfo()